In [6]:
!pip install pandas #판다스 설치
!pip install tqdm #막대바 설치 (얼마나 걸리는지 확인을 위하여)

  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.66.4-py3-none-any.whl (78 kB)


In [7]:
# 필요한 립 임포트
import time #타임슬립 립
import pandas as pd # 데이터 분석을 위한 판다스 모듈
import re # 정규 표현식 모듈 '특정 패턴'찾기 혹은 조작=검색 언어
from selenium.webdriver.common.by import By # 셀레니움에서 요소를 찾기 위한 By 클래스
from selenium.webdriver.support.ui import WebDriverWait #셀레니움에서 명시적 대기를 위한 클래스
from selenium.webdriver.support import expected_conditions as EC #셀레니움에서 명시적 "대기 조건"을 위한 expected_conditions 모듈 예상상태 위와 1SET
from selenium.common.exceptions import TimeoutException #타임아웃 예외 처리
from tqdm import tqdm #진행 상황을 표시하기 위한 tqdm 모듈


In [10]:
filename = r'C:\[lauren]\5_Project\filename.csv' #filename경로를 찾지못해 경로 설정.
columns_to_check = ['주연1','주연2','주연3','주연4','줄거리','러닝타임','포스터'] #체크할 칼럼즈 목록 정의.
df_combined = pd.read_csv(filename, encoding='utf-8') #데이터 프레임으로 filename CSV읽어오기. 

def filter_rows_with_missing_values(filename): #가장 (1)큰 함수 '결측값이 있는 행 필터링'
    movies_to_update = [] #업데이트 할 영화목록 빈 리스트 생성

    for index, row in df_combined.iterrows(): #df_combined에서 iterrow를 사용하여 각 행,인덱스를 차례대로 순회.
#.iterrows메소드는 "각 행을 인덱스와 함께 반복" 'ndex'변수에 각 행의 인덱스 할당, 
#'row'변수에는 해당 행의 데이터가 시리즈 형태로 할당.
        for column in columns_to_check: #columns_to_check에서 column을 차례로 반복하여 (체크할 각 칼럼에 대해 확인)
            if row[column] == '없음': #해당 칼럼에 '없음'이 있으면,
                mName = row['Name'] #mName에 행'Name'할당
                mDirector =row['Director1'] #mDirector에 행 Director1 할당.
                movies_to_update.append((mName,mDirector)) #비워둔 movie_to_update에 새로만든 mName,mDirector 행 추가
            break #내부 for 루프 종료하고 다음 행으로 이동


    if movies_to_update: #만약 movies_to_update에 
        df_combined = search_and_extract_movie_info(movies_to_update,df_combined)
    #movies_to_update,df_combined을 영화정보에 찾고 추출하여 df_combined 할당
    #(2)search_and_extract_movie_info **보조함수** movies_to_update,df_combined를 할당.
    #(movies_to_update,df_combined) 둘다 search_and_extract_movie_info의 인자 df_combined는 데이터 프레임을 말하고, 
    #df_combined 변수에 저장
    
    return df_combined #업데이트된 df_combined에 반환 

# filter_rows_with_missing_values 함수는 주어진 CSV 파일에서 '없음' 값이 있는 행을 찾고, 필요한 추가 작업이 있다면 
# search_and_extract_movie_info 함수를 호출하여 해당 정보를 업데이트하는 역할을 담당합니다. 
# search_and_extract_movie_info 함수는 filter_rows_with_missing_values 함수에 의해 호출되어 사용되는 보조 함수입니다.

from IPython.display import display #(민영추가) IPython.display 모듈의 display 함수를 사용하는 모듈 임포트 주피터에서만
display(df_combined) #통하여 데이터 프레임으로 보게 함. 
print(df_combined) # 업데이트된 df_combined에 반환된것 보기좋게 데이터 프레임으로 확인 

###################첫번째 함수 ok#####################

def search_and_extract_movie_info(movies_to_update,df_combined):#movies_to_update,df_combined의 영화를 찾고 추출하는 함수 
    driver = webdriver.Chrome() #크롬 웹 드라이버 인스턴스 생성
    updated_movies = set() #집합함수 중복값 허용x , 없데이트 된 영화를 추적할 집합 초기화(?)

    try : # 예외처리 내용 
        total_movies = len(movies_to_update) #movies_to_update의 갯수를 total_movies에 할당 = 영화의 총 수 계산
        progress_bar = tqdm(total=total_movies,desc="진행중",position=0) #전체 반복량은 total_movies,진행바 앞에 텍스트"진행중"출력,시작값은 0으로 시작

        for movie_name,movie_director in movies_to_update: #movies_to_update에 movie_name,movie_director 반복
            if movie_name in update_movies: #만약  movie_name과 update_movies이 동일하면, 
                continue #업데이트 된 영화는 건더띈다.

            driver.get('https://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do') #코비스 영화 검색 페이지 열고

            movie_name_element = WebDriverWait(driver,10).until(
                EC.presence_of_element_located((By.XPATH,'//*[@id="searchForm"]/div[1]/div[1]/div/input'))
            ) # WebDriverWait을 통해 driver를 "최대" 10초동안 기다린다.언제까지? -> ID가 "searchForm"인 엘리먼트가 나올 때까지 
            #해당 엘리먼트가 나오면, EC에서 True를 리턴 즉, ID가 searchForm인 element가 나올 때까지 최대 10초를 기다린다.
            movie_name_element.click() # 요소가 나오면 클릭
            movie_name_element.send_keys(movie_name) # 영화 이름 입력 필드를 기다리고 요소 클릭 후,영화 이름 입력

            time.sleep(1) #1초 기다림. 

            movie_director_element = WebDriverWait(driver,10).until(
                EC.presence_of_element_located((By.XPATH,'//*[@id="searchForm"]/div[1]/div[2]/div/input'))
            )#웹드라이버를 통해 최대 10초까지 기다린 후 , id=searchForm 엘리먼츠가 나올때까지, 그리고 거기서 나오면 트루,
            movie_director_element.click() 
            mocie_director_element.send_keys(movie_director)
            
            #클릭 후, movie_director 영화 감독 입력 
            time.sleep(1)
            # 1초 대기

            ##여기까지가 해당 검색 페이지에서 영화 이름과, 감독이름 검색 후 클릭이다##
            movie_search_element = WebDriverWait(driver,10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="searchForm"]/div[1]/div[5]/button[1]'))
            ) #최대10초까지 기다리고 버튼 클릭.
            movie_search_element.click()
            # 검색 버튼을 기다려서 클릭

            time.sleep(5)
            # 5초 대기

            found_movie = False #found_movie 영화찾기 상태 초기화 

            for i in range(1,10) : #검색 결과 최대 9개까지 결과를 순회 
                try: #예외처리내용
                    movie_index_element = WebDriverWait(driver,3).until(
                        EC.presence_of_element_located((By.XPATH, f'//*[@id="content"]/div[4]/table/tbody/tr[{i}]/td[1]/span/a'))
                    ) #웹드라이버 통해서 최대 3초까지 id="content"의 tr[{i}]/td[1]/span/a 나올때까지 (9번순회)
                    if movie_index_element.text == movie_name: #만약 movie_index_element의 텍스트가 movie_name과 동일하면,
                        movie_index_element.click() #인덱스 요소 클릭 
                        found_movie = True # 상태 갱신 
                        break #빠져나감
                except Exception: #예외처리 
                    pass #넘기기

            if not found_movie: #found_movie가 아니면, 
                progress_bar.write(f"'{movie_name}'영화를 찾지 못했습니다.")
                continue
                #{movie_name}의 영화를 찾지못하면 진행바에 '영화를 찾지 못했습니다'메세지 출력 후 다음 영화 진행.
            movie_info = {'Name':movie_name,'Director1':movie_director}
                #영화 정보 초기화 (상태) movie_info에 딕셔너리 Name:movie_name, Director1:movie_director저장.

            for i in range(1,5): #주연배우 정보를 4번 반복하여 찾기. 
                try: #예외처리내용 
                    movie_actor_element = WebDriverWait(driver,10).until(
                        EC.presence_of_element_located((By.XPATH, f'/html/body/div[3]/div[2]/div/div[1]/div[*]/div/dl/div[2]/dd/table/tbody/tr/td/a[{i}]'))
                        #웹드라이버 최대 10초 동안 기다렷다가 i루프가 나올때까지 4번까지 반복되면서 주연배우4명 정보 가져옴.
                    )
                    actor_name = movie_actor_element.text.split('(')[0].strip()
                    #movie_actor_element의 text에서 (기준으로 잘라서 [0]주연 이름 첫번재를 공백기준으로 제거하여
                    movie_info[f'주연{i}'] = actor_name
                    # actor_name 주연 배우 정보를 movie_info{i}에 저장.
                except Exception: #예외처리내용에 대한 대응
                    movie_info[f'주연{i}']="없음"
                    #movie_info주연{i}배우가 나오지 않으면 "없음"으로 출력

            try : # 예외처리내용
                plot_element = None #줄거리를 None값으로 설정
                for div in range(1,11) #div에 1부터10번 반복
                    try : #예외처리내용
                        plot_element = WebDriverWait(driver,2).until(
                            #웹드라이버 통해서 최대 2초까지 .until 해당사항 먹힐때까지
                            EC.presence_of_element_located((By.XPATH, f'/html/body/div[3]/div[2]/div/div[1]/div[{div}]/p'))
                        )  #해당 페이지 {div}에 10회 반복하여 
                        if plot_element: #만약plot_element가 true이면
                            movie_info['줄거리'] = plot_element.text
                            # plot_element 줄거리 정보를 추출하여 movie_info['줄거리']에 추가
                            break #빠져나감
                    except TimeoutException: #예외대응으로 시간내에 추출하지 못하면 
                        continue #계속 진행

                        if not plot_elment: #plot_elment가 비어있으면
                            movie_info['줄거리'] = "줄거리 정보 없음" #'줄거리'를 찾지 못하면 "줄거리 정보 없음"출력
                    except Exception: #예외대응 
                        movie_info['줄거리'] = "줄거리 정보 없음" #"줄거리 정보 없음"으로 출력
                    
                    try: #예외처리내용
                        runtime_element = WebDriverWait(driver,10).until(
                            EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[2]/div/div[1]/div[2]/dl/dd[4]'))
                        )#최대 10초 동안 해당페이지 요소 출력
                        img_src = img_element.get_attribute('src') #접속하여 'src' 속성을 찾아 img_src에 저장
                        movie_info['포스터'] = img_src #img_src를 movie_info['포스터']에 저장.
                    except Exception: #예외대응 
                        movie_info['포스터'] = "사진 링크 없음"
                        # 포스터를 찾지 못하면 "사진 링크 없음"로 출력

                    for column in movie_info.keys(): #칼럼에 movie_info딕셔너리의 모든키값을 반복.
                        if column in df_conbined.columns and \ #만약 column에 df_combined의 열 이름에 포함되어 있고,
                        df_combined.loc[df_combined['Name']]==movie_name,column].values[0] == '없음':                        
                        #df_combined 데이터프레임에서 'Name' 열의 값이[0] movie_name과 
                        #같은 행의 column 열 값이 '없음'인 경우
                            df_combined.loc[df_combined['Name'] == movie_name, column] = movie_info[column]
                            # df_combined 데이터프레임에서 'Name' 열의 값이 movie_name과 (이름)같은 행의 column 열에 
                            #movie_info[column] 값을 저장.
                            ## 결국 '없음'으로 설정된 칼럼을 업데이트 
                    
                    updated_movies.add(movie_name)
                    # 업데이트 된 영화로 추가
                    progress_bar.update(1) #진행바에 1%씩 증가.

                    df_combined.to_csv('movie_take_new(test).csv'),index=False,encoding='utf-8'
                    #익덱스 없이 인코딩 utf-8로 df_combined을 'movie_take_new(test).csv' csv로 저장

                 df_combined.to_csv('movie_take_new(test).csv', index=False, encoding='utf-8')
                    # 최종 저장

                finally : #최종
                    time.sleep(3)  #3초 기다렸다가
                    driver.quit() #드라이버 종료
                
                return df_combined # 업데이트된 df_combined 반환

            movie_info_df = filter_rows_with_missing_values('movie_data_test.csv')
            # 'movie_data_test.csv' 파일에서 결측값이 있는 행을 필터링하여 movie_info_df에 저장





















SyntaxError: incomplete input (1410361218.py, line 107)

In [11]:
print(df_combined)

      Unnamed: 0          Name                         Name(En)  Year Country  \
0              0           더 문                         The Moon  2023      한국   
1              1       타임코드 57                       57 Seconds  2023      미국   
2              2         유랑지구2            The Wandering Earth 2  2023      중국   
3              3         크리에이터                      The Creator  2023      미국   
4              4  믿거나 말거나, 진짜야              Incredible but True  2022     프랑스   
...          ...           ...                              ...   ...     ...   
4583        4583      비바 라스베가스                    Viva Lasvegas  1964      미국   
4584        4584   웨스트 사이드 스토리                  West Side Story  1961      미국   
4585        4585         장미의 곡                   A Song Of Rose  1960      한국   
4586        4586        7인의 신부  Seven Brides For Seven Brothers  1954      미국   
4587        4587         푸른 언덕                    The Blue Hill  1948      한국   

     Genre1 Genre2  Genre3 

In [3]:
for i in range(1,11):
    print(i)

1
2
3
4
5
6
7
8
9
10
